1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [1]:
import numpy as np
import csv
from random import randrange
from os import system

# 1 crete and save data_int.txt
list = []
for i in range(100):
    list.append(randrange(100))
print(list)

txt_file = open("data_int.txt", "w")
for element in list:
    txt_file.write(str(element) + " ")
txt_file.close()

system("cat data_int.txt")

# 5x5 floats
matrix = np.random.rand(5, 5)
txt_file2 = open("data_float.txt", "w")
for element in matrix:
    txt_file2.write(str(element) + " ")
txt_file2.close()
system("cat data_float.txt")

# txt convert to csv
with open('data_float.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('data_float.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerows(lines)
system("cat data_float.csv")


[18, 63, 53, 17, 12, 93, 32, 6, 15, 90, 54, 13, 67, 37, 46, 85, 20, 77, 16, 24, 75, 32, 94, 0, 25, 4, 73, 55, 59, 79, 74, 23, 40, 1, 84, 4, 7, 42, 73, 51, 12, 56, 36, 36, 61, 46, 30, 0, 91, 91, 31, 58, 68, 37, 9, 30, 3, 81, 90, 23, 65, 13, 77, 80, 5, 15, 29, 20, 92, 24, 36, 36, 8, 58, 27, 28, 12, 42, 36, 80, 77, 0, 19, 46, 74, 57, 3, 1, 52, 48, 61, 62, 92, 4, 0, 56, 76, 72, 2, 18]
18 63 53 17 12 93 32 6 15 90 54 13 67 37 46 85 20 77 16 24 75 32 94 0 25 4 73 55 59 79 74 23 40 1 84 4 7 42 73 51 12 56 36 36 61 46 30 0 91 91 31 58 68 37 9 30 3 81 90 23 65 13 77 80 5 15 29 20 92 24 36 36 8 58 27 28 12 42 36 80 77 0 19 46 74 57 3 1 52 48 61 62 92 4 0 56 76 72 2 18 [0.80209279 0.50030616 0.50436263 0.23041311 0.58946038] [0.04072945 0.84779539 0.49588382 0.95414215 0.133845  ] [0.79891986 0.13937443 0.33500964 0.69740628 0.32759495] [0.87219892 0.37213362 0.83468195 0.60371548 0.62862985] [0.71264997 0.31749289 0.04398877 0.15893742 0.46061089] [0.80209279 0.50030616 0.50436263 0.23041311 0.5

0

2\. **JSON files**

Load the file `user_data.json`, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [3]:
import pandas as pd
file = pd.read_json("user_data.json")
print(file)
file = file[(file['CreditCardType'] == "American Express")]
print(file)

      ID             JobTitle                    EmailAddress  \
0      1      Project Manager    Remy_Crawford1065@typill.biz   
1      2  Investment  Advisor    Clint_Thorpe5003@bulaffy.com   
2      3                 Cook   Russel_Hooper9403@bulaffy.com   
3      4      Business Broker   Benjamin_Rowan6280@gembat.biz   
4      5         Loan Officer       Matt_Lloyd3241@guentu.biz   
..   ...                  ...                             ...   
195  196         Loan Officer        Wade_Khan6086@twipet.com   
196  197                 Cook         Ally_Rose8531@twace.org   
197  198     Machine Operator      Harvey_Moore7505@extex.org   
198  199  Investment  Advisor      Ryan_Kennedy5565@corti.com   
199  200          Stockbroker  Michaela_Murphy8359@ovock.tech   

    FirstNameLastName           CreditCard    CreditCardType  
0       Remy Crawford  6205-7258-7671-0612       Capital One  
1        Clint Thorpe  7083-8766-0251-2345  American Express  
2       Russel Hooper  6378-44

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [4]:
import pandas as pd
from os import system
data = pd.read_csv("mushrooms_categorized.csv")
data =data.groupby('class').mean()
print(data)
data.to_json('mushrooms_categorized.json')

system("cat mushrooms_categorized.json")

       cap-shape  cap-surface  cap-color   bruises      odor  gill-attachment  \
class                                                                           
0       3.266160     1.615970   4.581749  0.653992  4.334601         0.954373   
1       3.436159     2.055158   4.421859  0.159346  3.940756         0.995403   

       gill-spacing  gill-size  gill-color  stalk-shape  ...  \
class                                                    ...   
0          0.285171   0.068441    6.622624     0.615970  ...   
1          0.028601   0.567926    2.863636     0.514811  ...   

       stalk-surface-below-ring  stalk-color-above-ring  \
class                                                     
0                      1.798479                6.098859   
1                      1.394280                5.512768   

       stalk-color-below-ring  veil-type  veil-color  ring-number  ring-type  \
class                                                                          
0                    

0

4\. **Reading a database**

Get the database `sakila.db` from the lecture `06_dataio.ipynb`, and import the table `actors` as a Pandas dataframe. Using the dataframe, count how many actors have a first name that begins with `A`.

*Hint:* use the Series `.str` method to apply the Python string methods to the elements of a Series, see [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.html).

5\. **Reading the credit card numbers**

Get the binary file named `credit_card.dat` from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [6]:
import pandas as pd
file_name = 'credit_card.dat'
credit_cards = []
with open(file_name, mode='r') as f:
    for line in f:
        n = 6
        bin_chars = [line[i:i + n] for i in range(0, len(line) - 5, n)]

        card_number = ""
        for char in bin_chars:
            card_number += chr(int(char, 2))

        credit_cards.append(card_number)

print(credit_cards)

['7648 5673 3775 2271', '3257 8247 3354 2266', '2722 0001 4011 6652', '0661 3063 3742 3150', '0432 1608 1462 4742', '5827 2027 8785 7303', '5774 8528 2087 1117', '8140 1210 6352 2845', '5764 1133 7301 7100', '6456 1737 4126 6726', '1228 8631 7382 0000', '7051 0160 5374 3166', '0618 3587 1630 6376', '1545 5454 7444 5636', '6735 3116 3202 6834', '7287 5011 1547 8413', '7033 2607 3328 4200', '2568 5244 1874 5024', '1684 2253 7570 7118', '0672 2576 0575 6631', '6332 8353 8787 1340', '1813 3361 1175 4211', '2477 6450 8840 2368', '5512 3505 2563 1326', '3083 7882 0621 0025', '4521 5148 8045 0334', '7563 3654 8713 5787', '8324 2664 0476 5561', '0565 2504 7168 3510', '5107 5507 1767 0738', '2462 1821 2448 1443', '2788 0638 6861 6554', '5851 5873 5474 0547', '0670 1004 4013 2655', '5874 5506 3048 0806', '2805 5401 8462 1260', '5083 8406 6310 1862', '1076 1445 3013 2266', '8440 4804 4844 5277', '4758 6141 0686 1387', '7586 0675 0315 2568', '2544 1258 7432 5165', '3474 5023 4434 5626', '1410 0270

6\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [ ]:
from IPython.display import Image
Image("images/data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, "pack" the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_dataio.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?